# Recognizing 2 and 5 in MNIST Dataset

In [28]:
import os
import matplotlib.pyplot as plt
from matplotlib.image import imread
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from NeuralNetworks.code.layers.fullyconnected import FC
from NeuralNetworks.code.layers.convolution2d import Conv2D
from NeuralNetworks.code.layers.maxpooling2d import MaxPool2D
from NeuralNetworks.code.activations.activations import get_activation
from NeuralNetworks.code.losses.meansquarederror import MeanSquaredError
from NeuralNetworks.code.losses.binarycrossentropy import BinaryCrossEntropy
from NeuralNetworks.code.models.model import Model
from NeuralNetworks.code.optimizers.adam import Adam
from NeuralNetworks.code.optimizers.gradientdescent import GD

print('libs imported successfully!')

libs imported successfully!


## Data loading
importing images

In [62]:

current_directory = os.getcwd()
mnist_folder = os.path.join(current_directory, "MNIST")

print("MNIST folder path:", mnist_folder)

MNIST folder path: C:\Users\Samin\Desktop\University\Term 7\Computational Intelligence\Projects\NeuralNetworks\code\datasets\MNIST


In [63]:
image_paths = []
for folder_name in os.listdir(mnist_folder):
    print(f'Opening folder {folder_name}')
    folder_path = os.path.join(mnist_folder, folder_name)
    if os.path.isdir(folder_path):
        image_paths.extend([os.path.join(folder_path, file_name) for file_name in os.listdir(folder_path)])
    else:
        print(f"{folder_path} is not a directory.")
print('Done')

Opening folder 2
Opening folder 5
Done


In [64]:
data = []
labels = []

In [65]:
def load_images_and_labels(image_paths):
    for image_path in image_paths:
        image = imread(image_path)
        if int(os.path.basename(os.path.dirname(image_path))) == 2:
           label = 0
        else:
           label = 1
           # Assuming the parent folder name represents the label
        data.append(image)
        labels.append(label)
    print('Images loaded and labeled successfully')

In [66]:
load_images_and_labels(image_paths)

Images loaded and labeled successfully


### Converting to dataset

In [67]:
data = np.array(data)
labels = np.array(labels)

In [68]:
data.shape

(2000, 28, 28)

In [69]:
labels.shape

(2000,)

In [70]:
flattened_data = data.reshape(data.shape[0], -1)  # Reshape to (2000, 784)

In [71]:
dataset = np.column_stack((flattened_data, labels))


In [72]:
df = pd.DataFrame(dataset, columns=[f"pixel_{i}" for i in range(flattened_data.shape[1])] + ["label"])


In [73]:
df.to_csv("mnist_dataset.csv", index=False)

PermissionError: [Errno 13] Permission denied: 'mnist_dataset.csv'

## Exploring dataset

In [3]:
df = pd.read_csv("mnist_dataset.csv")
num_rows, num_cols = df.shape
print("Number of rows:", num_rows)
print("Number of columns:", num_cols)

Number of rows: 2000
Number of columns: 785


In [4]:
df.head()

,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,...,pixel_775,pixel_776,pixel_777,pixel_778,pixel_779,pixel_780,pixel_781,pixel_782,pixel_783,label
0,3,0,10,0,0,3,0,2,7,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,5,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,7,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df.describe()

,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,...,pixel_775,pixel_776,pixel_777,pixel_778,pixel_779,pixel_780,pixel_781,pixel_782,pixel_783,label
count,2000.00000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.0000,2000.000000,2000.000000,...,2000.00000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,0.36350,0.463000,0.546500,0.349500,0.489000,0.460500,0.410500,0.4355,1.917500,2.334500,...,0.21450,0.151000,0.281000,0.228000,0.209000,0.004500,0.002500,0.003500,0.003500,0.500000
std,1.47833,1.829564,1.973014,1.511123,1.864301,1.882342,1.794882,1.6650,3.052434,3.964779,...,1.25908,0.906427,1.381663,1.216041,1.215343,0.201246,0.111803,0.156525,0.120395,0.500125
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000
75%,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,3.000000,3.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,16.00000,17.000000,17.000000,18.000000,19.000000,23.000000,31.000000,16.0000,18.000000,24.000000,...,20.00000,12.000000,19.000000,18.000000,16.000000,9.000000,5.000000,7.000000,5.000000,1.000000


In [6]:
label_counts = df['label'].value_counts()
label_counts

label
0    1000
1    1000
Name: count, dtype: int64

### Creating test and train

In [21]:
# Separate features (pixels) and labels
X = df.drop('label', axis=1)  # Features (pixel columns)
y = df['label']  # Labels

#### Normalize the features

In [29]:
scaler = MinMaxScaler()
# Fit the scaler to your data
scaler.fit(X)
# Apply the normalization to your data
X_normalized = scaler.transform(X)

In [30]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

In [31]:
X_train.shape

(1600, 784)

In [32]:
X_train

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.05882353, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [33]:
X_test.shape

(400, 784)

In [34]:
X_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Reshape the input data to a 4D tensor

In [37]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)
X_train.shape

(1600, 28, 28, 1)

### Defining Architecture

In [38]:
arch = {
    'Conv1': Conv2D(in_channels=1, out_channels=32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), name='Conv1'),
    'MaxPool1': MaxPool2D(kernel_size=(2, 2), stride=(2, 2), mode='max'),
    'Conv2': Conv2D(in_channels=32, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), name='Conv2'),
    'MaxPool2': MaxPool2D(kernel_size=(2, 2), stride=(2, 2), mode='max'),
    'Conv3': Conv2D(in_channels=64, out_channels=128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), name='Conv3'),
    'MaxPool3': MaxPool2D(kernel_size=(2, 2), stride=(2, 2), mode='max'),
    'Conv4': Conv2D(in_channels=128, out_channels=256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), name='Conv4'),
    'MaxPool4': MaxPool2D(kernel_size=(2, 2), stride=(2, 2), mode='max'),
    'Activation1': get_activation('relu'),
    'FC1': FC(input_size=256, output_size=1, name='FC1'),
    'Activation2': get_activation('sigmoid'),
}


#### Create the criterion (loss) function

In [39]:
criterion = BinaryCrossEntropy()

#### Create the optimizer

In [40]:
optimizer = Adam(arch, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8)


#### Creating Model

In [41]:
model = Model(arch, criterion, optimizer)

In [42]:
epochs = 10
batch_size = 32

In [43]:
train_loss, val_loss = model.train(X_train, y_train, epochs, val=None, batch_size=batch_size, shuffling=True, verbose=1)

TRAINING DATA STARTED...


  0%|          | 0/10 [00:00<?, ?it/s]

EPOCH = 1 
ONE EPOCH STARTED...
MODEL FORWARD STARTING
layer_name : Conv1
layer detected
IN CNN FORWARD
CNN FORWARD DONE
Z is added
layer_name : MaxPool1
layer detected
MAX POOLING FORWARD STARTED...
MAX POOLING FORWARD ENDED
Z is added
layer_name : Conv2
layer detected
IN CNN FORWARD
CNN FORWARD DONE
Z is added
layer_name : MaxPool2
layer detected
MAX POOLING FORWARD STARTED...
MAX POOLING FORWARD ENDED
Z is added
layer_name : Conv3
layer detected
IN CNN FORWARD
CNN FORWARD DONE
Z is added
layer_name : MaxPool3
layer detected
MAX POOLING FORWARD STARTED...
MAX POOLING FORWARD ENDED
Z is added
layer_name : Conv4
layer detected
IN CNN FORWARD
CNN FORWARD DONE
Z is added
layer_name : MaxPool4
layer detected
MAX POOLING FORWARD STARTED...
MAX POOLING FORWARD ENDED
Z is added
layer_name : Activation1
activation detected
layer_name : FC1
layer detected
FC Forward Step:
input shape: (32, 1, 1, 256)
yes to conv
FC forward done!
Z is added
layer_name : Activation2
activation detected
MODEL FOR

 10%|█         | 1/10 [15:53<2:23:02, 953.57s/it]

MAX POOLING BACKWARD ENDED
Conv1
layer detected
IN CNN BACKWARD
CNN BACKWARD DONE
MODEL BACKWARD ENDED
UPDATING PARAMETERS
IN ADAM
IN ADAM
IN ADAM
IN ADAM
IN ADAM
Epoch 1: train cost = 34.676004148112725
EPOCH = 2 
ONE EPOCH STARTED...
MODEL FORWARD STARTING
layer_name : Conv1
layer detected
IN CNN FORWARD
CNN FORWARD DONE
Z is added
layer_name : MaxPool1
layer detected
MAX POOLING FORWARD STARTED...
MAX POOLING FORWARD ENDED
Z is added
layer_name : Conv2
layer detected
IN CNN FORWARD
CNN FORWARD DONE
Z is added
layer_name : MaxPool2
layer detected
MAX POOLING FORWARD STARTED...
MAX POOLING FORWARD ENDED
Z is added
layer_name : Conv3
layer detected
IN CNN FORWARD
CNN FORWARD DONE
Z is added
layer_name : MaxPool3
layer detected
MAX POOLING FORWARD STARTED...
MAX POOLING FORWARD ENDED
Z is added
layer_name : Conv4
layer detected
IN CNN FORWARD
CNN FORWARD DONE
Z is added
layer_name : MaxPool4
layer detected
MAX POOLING FORWARD STARTED...
MAX POOLING FORWARD ENDED
Z is added
layer_name : 

...

MAX POOLING BACKWARD ENDED
Conv1
layer detected
IN CNN BACKWARD
CNN BACKWARD DONE
MODEL BACKWARD ENDED
UPDATING PARAMETERS
IN ADAM
IN ADAM
IN ADAM
IN ADAM
IN ADAM
Epoch 8: train cost = 35.019987928188634
EPOCH = 9 
ONE EPOCH STARTED...
MODEL FORWARD STARTING
layer_name : Conv1
layer detected
IN CNN FORWARD
CNN FORWARD DONE
Z is added
layer_name : MaxPool1
layer detected
MAX POOLING FORWARD STARTED...
MAX POOLING FORWARD ENDED
Z is added
layer_name : Conv2
layer detected
IN CNN FORWARD
CNN FORWARD DONE
Z is added
layer_name : MaxPool2
layer detected
MAX POOLING FORWARD STARTED...
MAX POOLING FORWARD ENDED
Z is added
layer_name : Conv3
layer detected
IN CNN FORWARD
CNN FORWARD DONE
Z is added
layer_name : MaxPool3
layer detected
MAX POOLING FORWARD STARTED...
MAX POOLING FORWARD ENDED
Z is added
layer_name : Conv4
layer detected
IN CNN FORWARD
CNN FORWARD DONE
Z is added
layer_name : MaxPool4
layer detected
MAX POOLING FORWARD STARTED...
MAX POOLING FORWARD ENDED
Z is added
layer_name : 

 90%|█████████ | 9/10 [2:30:01<16:39, 999.43s/it]

MAX POOLING BACKWARD ENDED
Conv1
layer detected
IN CNN BACKWARD
CNN BACKWARD DONE
MODEL BACKWARD ENDED
UPDATING PARAMETERS
IN ADAM
IN ADAM
IN ADAM
IN ADAM
IN ADAM
Epoch 9: train cost = 35.06549818316404
EPOCH = 10 
ONE EPOCH STARTED...
MODEL FORWARD STARTING
layer_name : Conv1
layer detected
IN CNN FORWARD
CNN FORWARD DONE
Z is added
layer_name : MaxPool1
layer detected
MAX POOLING FORWARD STARTED...
MAX POOLING FORWARD ENDED
Z is added
layer_name : Conv2
layer detected
IN CNN FORWARD
CNN FORWARD DONE
Z is added
layer_name : MaxPool2
layer detected
MAX POOLING FORWARD STARTED...
MAX POOLING FORWARD ENDED
Z is added
layer_name : Conv3
layer detected
IN CNN FORWARD
CNN FORWARD DONE
Z is added
layer_name : MaxPool3
layer detected
MAX POOLING FORWARD STARTED...
MAX POOLING FORWARD ENDED
Z is added
layer_name : Conv4
layer detected
IN CNN FORWARD
CNN FORWARD DONE
Z is added
layer_name : MaxPool4
layer detected
MAX POOLING FORWARD STARTED...
MAX POOLING FORWARD ENDED
Z is added
layer_name : 

100%|██████████| 10/10 [2:46:09<00:00, 996.96s/it]

MAX POOLING BACKWARD ENDED
Conv1
layer detected
IN CNN BACKWARD
CNN BACKWARD DONE
MODEL BACKWARD ENDED
UPDATING PARAMETERS
IN ADAM
IN ADAM
IN ADAM
IN ADAM
IN ADAM
Epoch 10: train cost = 35.10153498599731


#### Save the trained model

In [51]:
model.save('trained_model')

#### Load the saved model

In [56]:
saved_model = model.load_model('trained_model')

#### Predicting test set

In [58]:
y_pred = model.predict(X_test)

MODEL FORWARD STARTING
layer_name : Conv1
layer detected
IN CNN FORWARD
CNN FORWARD DONE
Z is added
layer_name : MaxPool1
layer detected
MAX POOLING FORWARD STARTED...
MAX POOLING FORWARD ENDED
Z is added
layer_name : Conv2
layer detected
IN CNN FORWARD
CNN FORWARD DONE
Z is added
layer_name : MaxPool2
layer detected
MAX POOLING FORWARD STARTED...
MAX POOLING FORWARD ENDED
Z is added
layer_name : Conv3
layer detected
IN CNN FORWARD
CNN FORWARD DONE
Z is added
layer_name : MaxPool3
layer detected
MAX POOLING FORWARD STARTED...
MAX POOLING FORWARD ENDED
Z is added
layer_name : Conv4
layer detected
IN CNN FORWARD
CNN FORWARD DONE
Z is added
layer_name : MaxPool4
layer detected
MAX POOLING FORWARD STARTED...
MAX POOLING FORWARD ENDED
Z is added
layer_name : Activation1
activation detected
layer_name : FC1
layer detected
FC Forward Step:
input shape: (400, 1, 1, 256)
yes to conv
FC forward done!
Z is added
layer_name : Activation2
activation detected
MODEL FORWARD ENDED


### Evaluate the performance on the test set

In [64]:
y_array_test = np.array(y_test)  # Convert list to array
y_array_2d_test = y_array_test[:, np.newaxis]  # Convert 1D array to 2D array (nx1)
y_array_2d_test.shape

(400, 1)

In [65]:
test_loss = criterion.compute(y_pred, y_array_2d_test)
print('Test Loss:', test_loss)

THIS IS BINAARY CROSS ENTROPY
batch size : 400
Test Loss: 0.7033890262365887
